In [2]:
import requests
import pandas as pd

In [24]:
url = 'https://opentdb.com/api.php?amount=20'

In [25]:
r = requests.get(url)
jokes = r.json()['results']
jokes = pd.DataFrame(jokes)
jokes.to_csv('jokes_df.csv', index=True)